# evaluate results

## count generated trajectories

In [2]:
import os
import json


data_path = 'data/json_2.1.0_Recep:original-bert'
feat_path = 'data/json_feat_2.1.0'
for split in ['valid_seen', 'valid_unseen']:
    cnt = 0
    for ep in os.listdir(os.path.join(data_path, split)):
        if os.path.isfile(os.path.join(data_path, split, ep)):
            continue
        for trial in os.listdir(os.path.join(data_path, split, ep)):
            for file in os.listdir(os.path.join(feat_path, ep, trial, 'pp')):
                if file.endswith('.json'):
                    cnt += 1
    print(cnt)

654
651


## Examination

### compare generation fail cases

In [9]:
import json
fail = {}
with open('gen/exp/0.3/fail-valid_seen.json') as f:
    recep = json.load(f)
with open('gen/exp/0.3/fail-valid_seen-pddlMatched.json') as f:
    recep_matched = json.load(f)
import pprint
pp = pprint.PrettyPrinter()

for key in recep_matched:
    if key not in recep:
        fail.update({key: recep_matched[key]})
    
total = 0  
cnt = 0
for trial, error in fail.items():
    total += 1
    if 'Empty plan' in error['error']:
        cnt += 1
print(total)
print(cnt)

66
18


### compare fail cases

이건 시뮬레이터로 실행시켜서 봐야 함

### compare generated pddl

In [4]:
import os
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

def process_task(raw_tasks):
    tasks = []
    for t in raw_tasks:
        if '\n' in t:
            tasks.extend([_t.replace('\b', '').strip() for _t in t.split('\n') if _t != ''])
        else:
            tasks.append(t.replace('\b', '').strip())
    return tasks

with open('gen/scripts/roberta-sentence_match.json', 'r') as f:
    sentence_match = json.load(f)
with open('gen/scripts/pddl_match.json') as f:
    prediction_pddl_match = json.load(f)
    
log = ''
data_path = 'data/json_2.1.0'
for split in ['valid_seen', 'valid_unseen']:
    for ep in os.listdir(os.path.join(data_path, split)):
        if os.path.isfile(os.path.join(data_path, split, ep)):
            continue
        for trial in os.listdir(os.path.join(data_path, split, ep)):
            with open(os.path.join(data_path, split, ep, trial, 'traj_data.json')) as f:
                traj_data = json.load(f)
            pddl = traj_data['pddl_params']
            tasks = process_task([ann['task_desc'] for ann in traj_data['turk_annotations']['anns']])
            predicted_pddl = [prediction_pddl_match[t] for t in tasks]
            
            for i, task in enumerate(tasks):
                if pddl == predicted_pddl[i]:
                    continue
                idx = sentence_match['pred'].index(task)
                log += os.path.join(ep, trial)+'\n'
                log += '%20s: %s\n' % ('valid', task)
                log += '%20s: %s\n' % ('matched train task', sentence_match['gt'][idx])
                log += '-'*50 + '\n'
                log += '%15s %15s %15s\n'%('params', 'predicted pddl', 'gt pddl')
                for p, v in pddl.items():
                    log += '%15s|%15s|%15s\n'%(p, predicted_pddl[i][p], v)
                log += '-'*50 + '\n\n\n'
with open('log-roberta.txt', 'w') as f:
    f.write(log)

# Utils

## make input.txt for eval trajs

In [6]:
input = []
for ep in os.listdir(os.path.join('data/json_2.1.0_Recep:original-gpt2/valid_seen')):
    if os.path.isfile(os.path.join('data/json_2.1.0_Recep:original-gpt2/valid_seen', ep)):
        continue
    for trial in os.listdir(os.path.join('data/json_2.1.0_Recep:original-gpt2/valid_seen', ep)):
        input.append(os.path.join('.', ep, trial))
        
with open(os.path.join('data/json_2.1.0_Recep:original-gpt2/valid_seen', 'input.txt'), 'w') as f:
    f.write('\n'.join(input))

## text 중복 제거

In [1]:
with open('found_valid_seen-recep:0.3.txt', 'r') as f:
    l = f.readlines()
l = list(set(l))
with open('found_valid_seen-recep:0.3.txt', 'w') as f:
    f.write(''.join(l))

# alfred unity simulator

initialization

In [4]:
from ai2thor.controller import Controller

controller = Controller(
    agentMode="locobot",
    visibilityDistance=1.5,
    scene="FloorPlan_Train1_3",
    gridSize=0.25,
    movementGaussianSigma=0.005,
    rotateStepDegrees=90,
    rotateGaussianSigma=0.5,
    renderDepthImage=False,
    renderInstanceSegmentation=False,
    width=300,
    height=300,
    fieldOfView=60
)
controller.reset(scene='FloorPlan_Train7_5', rotateStepDegrees=90)

<ai2thor.server.Event at 0x7f6415f427c0
    .metadata["lastAction"] = RotateRight
    .metadata["lastActionSuccess"] = True
    .metadata["errorMessage"] = "
    .metadata["actionReturn"] = None
>

agent actions

In [ ]:
controller.step(action='MoveAhead')
positions = controller.step(action='GetReachablePositions').metadata['actionReturn']

events
> metadata, frame, depth, instance_segmentation, instance_mask (+ bounding box)

In [ ]:
event = controller.step(action='rotateLeft')
# or
event = controller.last_event

shortest path

In [ ]:
from ai2thor.util.metrics import (
    get_shortest_path_to_object_type,
    path_distance,
    compute_single_spl
)

path = get_shortest_path_to_object_type(
    controller=controller,
    object_type='Apple',
    initial_position=dict(
        x=0.0,
        y=0.9,
        z=0.25
    )
)

path_distance(path)
compute_single_spl(path, shortest_path, successful_path)